# Step 2: Process high frequency GPS data
In this step, we will process the high frequency GPS data from the [Microsoft Geolife dataset](https://www.google.com/url?q=https%3A%2F%2Fwww.microsoft.com%2Fen-us%2Fdownload%2Fdetails.aspx%3Fid%3D52367), which contains high frequency GPS coordinates for 184 users in Beijing over five years. Trajectories of GPS coordinates are stored in `.plt` files for each individual.
After initial data ingestion, we take the following processing steps:
* Eliminate points outside of Beijing
* Get the hex id for each coordinate at the relevant resolution
* Merge/join the park ids for relevant hexes
* Calculate time between points so a total duration at each park can be calculated

The final output of this step is a dataframe of park visits for each individual and the length of their stay. We filter out stays less than 1 minute.




## 0. Set Up



### 0.1 Load packages

In [1]:
from google.colab import drive
import os
import gdown, zipfile
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import folium
from pathlib import Path
from collections import Counter
from statistics import mean, median, pstdev
import gc
import gdown

In [2]:
!pip install h3
import h3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00


### 0.2 Mount Your Google Drive
You will be asked for permission to access your Google Drive.

Note: The `h3_index_demo` folder must be downloaded and unzipped to your personal Google Drive in order to run this code in Colab.

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


### 0.3 Change project directory
If the project folder is not in your main directory (`/content/drive`), change the directory here by updating `my_dir`.


In [4]:
try:
  os.chdir('/content/drive/MyDrive/h3_index_demo')
  print('Successfully changed project directory')
except:
  print('Project not in main Drive directory')
  try:
    # Define your containing folder here if not in main Drive directory
    my_dir = '/content/drive/MyDrive/!data_science'
    os.chdir(my_dir + '/h3_index_demo')
    print('Successfully changed project directory')
  except:
    print('Could not change to project directory.\nDid you define your containing folder?')

Project not in main Drive directory
Successfully changed project directory


### 0.4 Variable definitions

In [5]:
# Data will be downloaded and unzipped locally to the vm rather than in Google drive
DATA_LOCAL = "/content/data/raw"
DATA_DIR = Path(DATA_LOCAL + "/Geolife Trajectories 1.3/Data")
Path(DATA_LOCAL).mkdir(parents=True, exist_ok=True)
# ETL and clean files are on Google Drive
DATA_ETL = Path("data/etl")
DATA_CLEAN = Path("data/clean")

# Beijing rough bounding box
BEIJING_MIN_LAT = 39.4
BEIJING_MAX_LAT = 41.1
BEIJING_MIN_LON = 115.4
BEIJING_MAX_LON = 117.6

In [6]:
# H3 resolution is 12, see previous step
RES = 12

### 0.5 Download and unzip data

In [7]:
# Use gdown package to download the data from Google Drive
file_id = "12p_IbESkP9YO4NwvAjtVZ66z8y65C4xZ"
out = Path(DATA_LOCAL + "/Geolife Trajectories 1.3.zip")
path = str(out)  # always define path

# Skip if already downloaded
if not out.exists() or out.stat().st_size == 0:
    url = f"https://drive.google.com/uc?id={file_id}"
    path = gdown.download(url, output=str(out), quiet=False, resume=True)
    if not path or not os.path.exists(path) or os.path.getsize(path) == 0:
        raise RuntimeError("Download failed")

Downloading...
From (original): https://drive.google.com/uc?id=12p_IbESkP9YO4NwvAjtVZ66z8y65C4xZ
From (redirected): https://drive.google.com/uc?id=12p_IbESkP9YO4NwvAjtVZ66z8y65C4xZ&confirm=t&uuid=33f3299b-6a57-4297-bd97-25d2132fc785
To: /content/data/raw/Geolife Trajectories 1.3.zip
100%|██████████| 313M/313M [00:05<00:00, 59.7MB/s]


In [8]:
# Unzip - this may take several minutes
zip_path = Path(path)                          # e.g. /content/data/raw/file.zip
unzip_dir = zip_path.parent / zip_path.stem  # e.g. /content/data/raw/file
extract_dir = zip_path.parent  # e.g. /content/data/raw

# Skip if that folder already exists and is non-empty
if unzip_dir.is_dir():
    print(f"Found existing folder: {extract_dir}. Skipping unzip.")
else:
    with zipfile.ZipFile(zip_path) as zf:
        zf.extractall(extract_dir)
    print(f"Unzipped to: {extract_dir}")

Unzipped to: /content/data/raw


## 1. Data exploration

### 1.1 View example file as pandas dataframe

**Geolife File Layout and Data**

Where files live
- `data/raw/Geolife Trajectories 1.3/Data/<user_id>/Trajectory/`
- `<user_id>` is a three-digit folder like `000`, `001`, `002`.
- Each trip is a `.plt` file named by its start time, e.g., `20081023025304.plt`.

What a `.plt` file contains
- First 6 lines are a header:
  - `Geolife trajectory`
  - `WGS 84`
  - `Altitude is in Feet`
  - other reserved lines
- After the header, each line is one GPS point with commas between values:
  - `lat, lon, 0, altitude_feet, excel_day_count, date, time`
- Latitude and longitude use WGS84.
- Altitude is in feet.
- Time appears as `date` and `time` strings

In [9]:
# Path to one trajectory file
user_id = "163"
fname = "20120710073254.plt"
fpath = DATA_DIR / user_id / "Trajectory" / fname

In [10]:
# Columns after the 6-line header in Geolife .plt files
# See also, the pdf User Guide
cols = [
    "lat",           # Latitude (deg)
    "lon",           # Longitude (deg)
    "unused",        # Always 0 in dataset - not sure the purpose
    "altitude_ft",   # Altitude in feet
    "days",          # Nnumber of days (with fractional part) that have passed since 12/30/1899
    "date",          # YYYY-MM-DD
    "time"           # HH:MM:SS
]

In [11]:
# Read in example file
example_df = pd.read_csv(
    fpath,
    skiprows=5,
    header=None,
    names=cols,
    dtype={
        "lat": "float64",
        "lon": "float64",
        "unused": "Int64",
        "altitude_ft": "float64",
        "days": "float64",
        "date": "string",
        "time": "string",
    }
)

In [12]:
example_df.head()

,lat,lon,unused,altitude_ft,days,date,time
0,0.000000,NaN,<NA>,NaN,NaN,<NA>,<NA>
1,39.986522,116.447732,0,490.856070,41100.314514,2012-07-10,07:32:54
2,39.986417,116.447905,0,487.486322,41100.314572,2012-07-10,07:32:59
3,39.986834,116.448105,0,151.543868,41100.314630,2012-07-10,07:33:04
4,39.986721,116.448311,0,149.812687,41100.314688,2012-07-10,07:33:09


In [13]:
# Build a proper timestamp
example_df["timestamp"] = pd.to_datetime(example_df["date"] + " " + example_df["time"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

### 1.1 Number of users and files

In [14]:
# Number of user and files
plt_paths = list(DATA_DIR.glob("*/Trajectory/*.plt"))
user_ids  = [p.parent.parent.name for p in plt_paths]
files_per_user = Counter(user_ids)

print(f"Total users: {len(files_per_user)}")
print(f"Total .plt files: {len(plt_paths)}")

Total users: 182
Total .plt files: 18670


In [15]:
# Summary statistics of files-per-user distribution
counts = list(files_per_user.values())
if counts:
    print(f"Files/user — mean: {mean(counts):.2f}, median: {median(counts)}, "
          f"min: {min(counts)}, max: {max(counts)}, sd: {pstdev(counts) if len(counts)>1 else 0.0:.2f}")

Files/user — mean: 102.58, median: 27.5, min: 1, max: 2153, sd: 249.38


## 2. Load data from .plt files
In a large-scale project, we might use pyspark / delta lake / databricks but for this demo, only pandas is needed.


### 2.1 Read in .plt files

In [16]:
# Read one Geolife .plt into minimal columns.
def load_one_plt(path: Path) -> pd.DataFrame:
    cols = ["lat","lon","unused","altitude_ft","excel_days","date","time"]

    df = pd.read_csv(
        path,
        skiprows=6,
        header=None,
        names=cols,
        usecols=[0,1,5,6],  # lat, lon, date, time
        dtype={"lat":"float64","lon":"float64","date":"string","time":"string"},
        na_values=["","NaN","nan"]
    )

    ts = pd.to_datetime(
        df["date"] + " " + df["time"],
        format="%Y-%m-%d %H:%M:%S",
        utc=True,
        errors="coerce"
    )

    user_id   = path.parent.parent.name                # .../Data/<user_id>/Trajectory/<file>.plt
    file_stem = path.stem                              # e.g., "20081023025304"
    traj_id   = f"{user_id}_{file_stem}"

    out = pd.DataFrame({
        "lat": df["lat"].astype("float64"),
        "lon": df["lon"].astype("float64"),
        "timestamp": ts,
        "user_id": user_id,
        "trajectory_id": traj_id
    })
    return out

In [17]:
# Read in and collect data from all files
frames = []
for p in tqdm(plt_paths, desc="Reading .plt files", unit="file"):
    try:
        frames.append(load_one_plt(p))
    except Exception as e:
        # Skip bad files; log if needed
        print(f"Failed to read {p}: {e}")

all_points = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["lat","lon","timestamp","user_id", "trajectory_id"])


Reading .plt files:   0%|          | 0/18670 [00:00<?, ?file/s]

### 2.2 Filter out points outside Beijing

In [18]:
# Drop invalid rows
all_points = all_points.dropna(subset=["lat","lon","timestamp"])
all_points = all_points[(all_points["lat"].between(-90, 90)) & (all_points["lon"].between(-180, 180))]


In [19]:
# Apply bbox
in_bj = (
    (all_points["lat"] >= BEIJING_MIN_LAT) & (all_points["lat"] <= BEIJING_MAX_LAT) &
    (all_points["lon"] >= BEIJING_MIN_LON) & (all_points["lon"] <= BEIJING_MAX_LON)
)
silver = all_points.loc[in_bj, ["user_id","trajectory_id","timestamp","lat","lon"]].copy()

In [20]:
# Derive dates
silver["event_date"]  = silver["timestamp"].dt.date

In [21]:
print("Rows parsed (valid):", len(all_points))
print("Rows in Beijing bbox:", len(silver))

Rows parsed (valid): 24876977
Rows in Beijing bbox: 20184273


In [23]:
# Clear all_points object from memory to save room
del all_points
# Clear list of chuncks also
try:
    frames.clear()
    del frames
except NameError:
    pass

gc.collect()

0

## 3. Get resolution 12 hex ids for GPS points
In the park data analysis, we selected a resolution of 12. We will now get the hex ids for all gps points.

In [24]:
silver.head()

,user_id,trajectory_id,timestamp,lat,lon,event_date
0,140,140_20080926105615,2008-09-26 10:56:15+00:00,39.978352,116.327377,2008-09-26
1,140,140_20080926105615,2008-09-26 10:56:16+00:00,39.978445,116.327367,2008-09-26
2,140,140_20080926105615,2008-09-26 10:56:18+00:00,39.978615,116.327350,2008-09-26
3,140,140_20080926105615,2008-09-26 10:56:20+00:00,39.978763,116.327340,2008-09-26
4,140,140_20080926105615,2008-09-26 10:56:22+00:00,39.978877,116.327333,2008-09-26


In [25]:
# Get number of duplicate latitude and longitude points to determine strategy for getting
# hex ids
n_rows = len(silver)
n_unique_xy = silver[["lat","lon"]].drop_duplicates().shape[0]
n_dupes = n_rows - n_unique_xy
pct_dupes = 100.0 * n_dupes / n_rows if n_rows else 0.0

print(f"Rows: {n_rows}")
print(f"Unique (lat,lon): {n_unique_xy}")
print(f"Duplicate rows by (lat,lon): {n_dupes} ({pct_dupes:.2f}%)")

Rows: 20184273
Unique (lat,lon): 18067438
Duplicate rows by (lat,lon): 2116835 (10.49%)


In [26]:
# Get h3 hex for each unique lat lon pair
unique_xy = silver[["lat", "lon"]].drop_duplicates().reset_index(drop=True)
unique_xy["hex_id"] = [
    h3.latlng_to_cell(lat, lon, RES)
    for lat, lon in unique_xy[["lat", "lon"]].to_numpy()
]


In [27]:
# Merge on hex_ids
silver = silver.merge(unique_xy, on=["lat", "lon"], how="left")

In [28]:
silver.head()

,user_id,trajectory_id,timestamp,lat,lon,event_date,hex_id
0,140,140_20080926105615,2008-09-26 10:56:15+00:00,39.978352,116.327377,2008-09-26,8c31aa50ead95ff
1,140,140_20080926105615,2008-09-26 10:56:16+00:00,39.978445,116.327367,2008-09-26,8c31aa50ead9dff
2,140,140_20080926105615,2008-09-26 10:56:18+00:00,39.978615,116.327350,2008-09-26,8c31aa50eadd7ff
3,140,140_20080926105615,2008-09-26 10:56:20+00:00,39.978763,116.327340,2008-09-26,8c31aa50eadd1ff
4,140,140_20080926105615,2008-09-26 10:56:22+00:00,39.978877,116.327333,2008-09-26,8c31aa50eadd9ff


## 4. Match locations to parks

### 4.1 Read in park hex file
File is unique on hex_id (one park per id).

In [29]:
park_hex_file = DATA_ETL / "park_hex_unique_r12.csv"

# Load park hex data
park_hex = pd.read_csv(
    park_hex_file, usecols=["osm_id", "hex_id"]
)

In [30]:
park_hex.head()

,osm_id,hex_id
0,836074671,8c31818041951ff
1,836074671,8c31818041959ff
2,836074671,8c3181804195dff
3,836074671,8c31818041a25ff
4,836074671,8c31818041a29ff


### 4.2 Merge on parks (osm_ids)

In [31]:
# Use the hex_id to merge on the parks (osm_ids)
silver = silver.merge(park_hex, on="hex_id", how="left")

In [32]:
# Number of GPS coordinates in parks
n_in_parks = silver["osm_id"].notna().sum()
print(f"Data points in parks: {n_in_parks}")

Data points in parks: 577808


### 4.3 Calculate total time spent at individual parks in a trajectory

In [33]:
# Create an indicator of whether the trajectory gps data contains any parks
silver["traj_has_park"] = (
    silver.groupby("trajectory_id")["osm_id"]
          .transform(lambda s: s.notna().any())
          .astype("int8")
)

In [34]:
# Keep only trajectories containing parks
park_traj = silver.loc[silver["traj_has_park"] == 1].copy()

In [35]:
# Number of trajectories containing parks
n_unique_traj = park_traj["trajectory_id"].nunique()
print(f"Unique trajectory_ids in park_traj: {n_unique_traj}")

Unique trajectory_ids in park_traj: 9088


In [36]:
# Ensure ordering within each trajectory
park_traj = park_traj.sort_values(["trajectory_id", "timestamp"])

In [37]:
# Create indicator for whether the previous row is in the same trajectory
prev_traj = park_traj["trajectory_id"].shift(1)

park_traj["same_traj_prev"] = (
    park_traj["trajectory_id"].eq(prev_traj).fillna(False).astype("int8")
)

In [38]:
# For all rows that have the same trajectory as previous, calculate the time delta in seconds
# Seconds since previous point in the SAME trajectory
prev_ts = park_traj.groupby("trajectory_id")["timestamp"].shift(1)
park_traj["delta_s"] = (park_traj["timestamp"] - prev_ts).dt.total_seconds()

# Keep deltas only where same_traj_prev == 1; else NaN
park_traj.loc[park_traj["same_traj_prev"] == 0, "delta_s"] = np.nan


In [39]:
# Distribution of duration
print(park_traj["delta_s"].describe())

count    1.616030e+07
mean     7.681909e+00
std      2.043993e+02
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      5.000000e+00
max      1.104770e+05
Name: delta_s, dtype: float64


In [40]:
# Create indicator for whether the previous row is in the same park and trajectory
# Previous park within trajectory
prev_osm = park_traj.groupby("trajectory_id")["osm_id"].shift(1)

park_traj["same_park_prev"] = (
    park_traj["osm_id"].notna() & prev_osm.notna() & (park_traj["osm_id"] == prev_osm) &
    park_traj["same_traj_prev"] == 1
).astype("int8")

In [41]:
# Get total time in park for each trajectory
# Note that you could require the time to be consecutive
# A full analysis should check for sensitivity to that choice
# Count only intervals that stay in the same park.
mask = (
    (park_traj["same_traj_prev"] == 1) &
    (park_traj["same_park_prev"] == 1) &
    park_traj["osm_id"].notna() &
    park_traj["delta_s"].notna()
)

time_by_traj_park = (
    park_traj.loc[mask]
             .groupby(["user_id", "trajectory_id", "osm_id"], as_index=False)["delta_s"]
             .sum()
             .rename(columns={"delta_s": "total_time_s"})
)

# Optional convenience fields
time_by_traj_park["total_time_min"] = time_by_traj_park["total_time_s"] / 60.0
time_by_traj_park["total_time_hr"]  = time_by_traj_park["total_time_s"] / 3600.0

print(time_by_traj_park.head())

  user_id       trajectory_id       osm_id  total_time_s  total_time_min  \
0     000  000_20081023025304  226936547.0           5.0        0.083333   
1     000  000_20081023025304  638620290.0          65.0        1.083333   
2     000  000_20081026134407  651001455.0          25.0        0.416667   
3     000  000_20081026134407  724597333.0          40.0        0.666667   
4     000  000_20081028003826   24827108.0        7235.0      120.583333   

   total_time_hr  
0       0.001389  
1       0.018056  
2       0.006944  
3       0.011111  
4       2.009722  


In [42]:
# Summary stats for minutes
dist = time_by_traj_park["total_time_min"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95,0.99])
print(dist.to_string())

count    15300.000000
mean         2.897093
std         20.894364
min          0.000000
10%          0.033333
25%          0.083333
50%          0.250000
75%          0.683333
90%          1.883333
95%          4.234167
99%         73.509333
max        822.950000


This GPS data is for trajectories/routes so most people are only passing through parks, possibly around the edges. The 90th percentile is a stay of 1.9 minutes. For the sake of this demo, I will use one minute as the cut point for a park visit, but there is much smarter logic that could be used given these data are for trajectories. For example, a trajectory that ends at a park might count as a stay, regardless of the duration. Ideally, this data would be of continuous location, rather than a trajectory.

In [43]:
# Indicator for stays longer than 1 minute
time_by_traj_park["stay_1min"] = (
    time_by_traj_park["total_time_min"].ge(1).fillna(False).astype("int8")
)

In [44]:
n_total = len(time_by_traj_park)
n_ge1 = int(time_by_traj_park["stay_1min"].sum())  # 0/1 int8
pct_ge1 = (n_ge1 / n_total * 100) if n_total else 0.0

print(f"Stays ≥1 min: {n_ge1} / {n_total} ({pct_ge1:.2f}%)")

Stays ≥1 min: 2901 / 15300 (18.96%)


In [45]:
# Unique users and parks with stays > 1 minute
mask = time_by_traj_park["total_time_min"] > 1

n_users = time_by_traj_park.loc[mask, "user_id"].nunique()
n_parks = time_by_traj_park.loc[mask, "osm_id"].nunique()

print(f"Users with >1 min stays: {n_users}")
print(f"Parks with >1 min stays: {n_parks}")

Users with >1 min stays: 126
Parks with >1 min stays: 387


In [46]:
# Filter and save rows that are at least one minute
mask = time_by_traj_park["total_time_min"] >= 1
out_path = DATA_CLEAN / "time_by_traj_park.csv"
time_by_traj_park.loc[mask].to_csv(out_path, index=False)
print(f"Wrote: {out_path.resolve()}  |  rows: {mask.sum()}")

Wrote: /content/drive/MyDrive/!data_science/h3_index_demo/data/clean/time_by_traj_park.csv  |  rows: 2901


In [47]:
time_by_traj_park.loc[mask].head()

,user_id,trajectory_id,osm_id,total_time_s,total_time_min,total_time_hr,stay_1min
1,000,000_20081023025304,638620290.0,65.0,1.083333,0.018056,1
4,000,000_20081028003826,24827108.0,7235.0,120.583333,2.009722,1
5,000,000_20081028003826,80716324.0,60.0,1.000000,0.016667,1
7,000,000_20081103232153,24827108.0,12245.0,204.083333,3.401389,1
9,000,000_20081111001704,24827108.0,7560.0,126.000000,2.100000,1
